In [1]:
sc

In [2]:
sc.version

u'2.1.0'

In [3]:
#lista de palabras
wordsList = ['spark', 'python', 'pypark', 'data','python']
wordsRDD = sc.parallelize(wordsList, 4)
# Print out the type of wordsRDD
print type(wordsRDD)

<class 'pyspark.rdd.RDD'>


In [4]:
result = wordsRDD.flatMap(lambda x: x.split(" ")).countByValue()
for key, value in result.iteritems():
    print "%s %i" % (key, value)

python 2
spark 1
pypark 1
data 1


#### ** Longitud de cada palabra **
#### Usamos `map()` y la función  `lambda`  para devolver el número de caracteres de cada palabra

In [5]:
wordsLengths = (wordsRDD.map(lambda x: len(x)).collect())
print wordsLengths

[5, 6, 6, 4, 6]


#### ** Pares RDDs **
#### El siguiente paso para escribir el contador de palabras es crear un nuevo tipo de RDD, llamado par RDD. Un par RDD es un RDD donde cada elemento es un par de tuplas `(k, v)` donde `k` es la clave y` v` es el valor. En este ejemplo, crearemos un par que consiste en `('<word>', 1)` para cada elemento de palabra en el RDD.
#### Podemos crear el par RDD usando la transformación `map ()` con una función `lambda ()` para crear un nuevo RDD.

In [6]:
wordPairs = wordsRDD.map(lambda x: (x, 1))
print wordPairs.collect()

[('spark', 1), ('python', 1), ('pypark', 1), ('data', 1), ('python', 1)]


#### Ahora, vamos a contar el número de veces que una palabra en particular aparece en el RDD. Hay varias maneras de realizar el conteo, pero algunas son más eficientes que otras.
#### Un posible enfoque sería hacer un  `collect ()` de todos los elementos y contarlos en el programa de controlador. Si bien este enfoque podría funcionar para pequeños conjuntos de datos, queremos un enfoque que funcione para cualquier tamaño de datos.

#### ** Usar `groupByKey()`  **
#### Una posible solución es usar [groupByKey()](http://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD.groupByKey). Como su nombre indica, la transformación `groupByKey ()` agrupa todos los elementos del RDD con la misma clave en una sola lista en una de las particiones. Hay dos problemas con el uso de `groupByKey ()`:
   + #### La operación requiere una gran cantidad de movimiento de datos para mover todos los valores a las particiones apropiadas.
   + #### No es óptimo para listas muy grandes.
 
#### Use `groupByKey ()` para generar un par RDD de tipo `('word', contador)`.

In [7]:
wordsGrouped = wordPairs.groupByKey()
for key, value in wordsGrouped.collect():
    print '{0}: {1}'.format(key, list(value))

python: [1, 1]
data: [1]
spark: [1]
pypark: [1]


#### El uso de la función `groupByKey ()` crea un RDD que contiene 3 elementos, cada uno de los cuales es un par de una palabra y un iterador de Python.
#### Ahora sume el iterador usando una transformación `map ()`. El resultado debe ser un par RDD que consiste en pares (palabra, contador).

In [8]:
wordCountsGrouped = wordsGrouped.map(lambda (k, v): (k, sum(v)))
print wordCountsGrouped.collect()

[('python', 2), ('data', 1), ('spark', 1), ('pypark', 1)]


#### ** Usando `reduceByKey` **
#### Una mejor solución es usar la función [reduceByKey()](http://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD.reduceByKey). La transformación `reduceByKey ()` reúne pares que tienen la misma clave y aplica la función proporcionada a dos valores a la vez, reduciendo iterativamente todos los valores a un solo valor. `ReduceByKey ()` funciona aplicando la función primero dentro de cada partición en una base per-key y luego a través de las particiones, permitiéndole escalar eficientemente a conjuntos de datos grandes.

In [9]:
wordCounts = wordPairs.reduceByKey(lambda x, y: x + y)
print wordCounts.collect()

[('python', 2), ('data', 1), ('spark', 1), ('pypark', 1)]


#### ** Todo junto **
#### La mejor versión es la que aplica la función `map()` sobre el RDD , aplica la transformación `reduceByKey()` , y al final llama a  `collect()`.

In [10]:
wordCountsCollected = (wordsRDD
                       .map(lambda x: (x, 1))
                       .reduceByKey(lambda x, y: x + y)
                       .collect())
print wordCountsCollected

[('python', 2), ('data', 1), ('spark', 1), ('pypark', 1)]


#### ** (3a) Palabras únicas **
#### De forma sencilla podemos obtener el número de palabras únicas

In [11]:
uniqueWords = wordCounts.count()
print uniqueWords

4


#### ** Capitalización y puntuación **
#### Al tratar con ficheros de texto es necesario tratar una serie de temas:
   + #### Las palabras deben ser contadas independientemente si están en mayúsculas o minúsculas
   + #### Eliminar signos de puntuación
   + #### Eliminar los espacios iniciales o finales de una línea.
 
#### Definimos la función `removePunctuation` que realiza estas tareas,convirtiendo todo el texto en minúsculas,eliminar signos de puntuación y espacios en blanco al principio y final de linea.  Para ello podemos usar el módulo de python  [re](https://docs.python.org/2/library/re.html)

In [12]:
import re
import string
def removePunctuation(text):
    """Removes punctuation, changes to lower case, and strips leading and trailing spaces.

    Note:
        Only spaces, letters, and numbers should be retained.  Other characters should should be
        eliminated (e.g. it's becomes its).  Leading and trailing spaces should be removed after
        punctuation is removed.

    Args:
        text (str): A string.

    Returns:
        str: The cleaned up string.
    """
    regex = re.compile('[%s]' % re.escape(string.punctuation))
    return regex.sub('', text).lower().strip()
print removePunctuation('Hi, you!')
print removePunctuation('No under_score!')

hi you
no underscore


### También podemos definir una función que elimine aquello que no queremos contabilizar

In [13]:
import re

def removePunctuation(text):
    return re.sub('[^a-z| |0-9]', '', text.strip().lower())

#### ** Cargar un fichero de texto **
#### Para convertir un archivo de texto en un RDD, usamos el método `SparkContext.textFile ()`. También aplicamos la función `removePunctuation ()`  usando una transformación `map ()` para eliminar la puntuación y cambiar todo el texto a minúsculas.

In [14]:
textRDD = (sc.textFile('about_spark.txt', 8).map(removePunctuation))

#### Antes de poder usar la función `wordcount ()`, tenemos que abordar dos problemas con el formato del RDD:
   + #### La primera cuestión es que necesitamos dividir cada línea por sus espacios.
   + #### La segunda cuestión es que necesitamos filtrar líneas vacías.
 
#### Aplicar una transformación que dividirá cada elemento del RDD por sus espacios. Para cada elemento del RDD, debe aplicar la función de cadena [split ()] (https://docs.python.org/2/library/string.html#string.split) de Python.

In [15]:
textWordsRDD = textRDD.flatMap(lambda x: x.split())
textWordCount = textWordsRDD.count()
textWordCount

205

In [16]:
print textWordsRDD.take(10)

[u'apache', u'spark', u'apache', u'spark', u'is', u'a', u'cluster', u'computing', u'platform', u'designed']


#### ** Contar las palabras **
#### A continuación vamos a detectar cual es la palabra que más veces aparece en el texto. Generaremos un ranking de las 10 más numerosas para que se vea parte del poder de spark.. Podemos ver las 10 palabras principales usando la acción `takeOrdered ()`

In [17]:
ranking= textWordsRDD.map(lambda x: (x,1)).reduceByKey(lambda y,z: y+z).takeOrdered(10,lambda x : -x[1])
ranking

[(u'the', 9),
 (u'and', 8),
 (u'spark', 8),
 (u'in', 8),
 (u'is', 7),
 (u'to', 6),
 (u'of', 4),
 (u'data', 4),
 (u'it', 4),
 (u'on', 3)]